### Import flow cytometry data and process with `fcsparser` parser library

In [1]:
import fcsparser
import os
from pathlib import Path

cwd = Path().resolve()
fcsBaseFn = "01-TripDay5-A1.fcs"
path = os.path.join(cwd, "data", fcsBaseFn)
meta, data = fcsparser.parse(path, meta_data_only=False, reformat_meta=True)
list(data)

['FSC-H',
 'FSC-A',
 'SSC-H',
 'SSC-A',
 'APC-H',
 'APC-A',
 'PB450-H',
 'PB450-A',
 'ECD-H',
 'ECD-A',
 'FSC-Width',
 'Time']

### Take a subset of the data

In [2]:
colsOfInterest = ['APC-H', 'PB450-H', 'ECD-H']
subset = data[colsOfInterest]
subset.head(n=5)

,APC-H,PB450-H,ECD-H
0,4840.399902,852.200012,649.500000
1,2291.800049,874.299988,594.599976
2,2495.600098,719.400024,509.700012
3,1660.599976,1273.099976,632.599976
4,2248.199951,890.900024,508.399994


### Log-transform the subset

In [3]:
import numpy as np

subsetLog10 = subset.apply(np.log10)
subsetLog10.head(n=5)

,APC-H,PB450-H,ECD-H
0,3.684881,2.930542,2.812579
1,3.360177,2.941660,2.774225
2,3.397175,2.856971,2.707315
3,3.220265,3.104862,2.801129
4,3.351835,2.949829,2.706206


### Render log-transformed subset into a 3D scatterplot via Plotly

In [4]:
import plotly
import plotly.io as pio
from plotly.offline import plot, init_notebook_mode
import plotly.graph_objs as go
import numpy as np
import copy

init_notebook_mode(connected=True)

#
# Base marker
#
baseMarker = {
    'color' : 'rgb(178, 190, 181)',
    'size' : 4,
    'symbol' : 'circle',
    'line' : {
        'color' : 'rgb(178, 190, 181)',
        'width' : 1
    },
    'opacity' : 0.1
}

#
# Highlight marker (+ attribute customizations)
#
highlightMarker = copy.deepcopy(baseMarker)
highlightMarker['color'] = 'rgb(229, 43, 80)'
highlightMarker['line']['color'] = 'rgb(229, 43, 80)'
highlightMarker['opacity'] = 1

#
# Plot boilerplate
#
def xyzPlot(x, y, z, name=None, marker=baseMarker, mode='markers', opacity=1):
    plot = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode=mode,
        marker=dict(
            color=marker['color'],
            size=marker['size'],
            symbol=marker['symbol'],
            line=marker['line'],
            opacity=marker['opacity']
        ),
        opacity=opacity,
        name=name
    )
    return plot

#
# Dataset
#
xCol, yCol, zCol = subsetLog10.iloc[:, 0], subsetLog10.iloc[:, 1], subsetLog10.iloc[:, 2]
x, y, z = np.array(xCol), np.array(yCol), np.array(zCol)
xRange, yRange, zRange = [0,6], [0,6], [0,6]

#
# Gate/filter values
#
gate = {
    'x' : 3.55,
    'y' : 3.2, 
    'z' : 3.02
}

figData = []

#
# Here, we use numpy array masks to filter the original dataset 
# by the gate values specified above.
#

# tripNegative ("highlighted")
xSub, ySub, zSub = x[x < gate['x']], y[y < gate['y']], z[z < gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="tripNegative", marker=highlightMarker))

# tripPositive
xSub, ySub, zSub = x[x >= gate['x']], y[y >= gate['y']], z[z >= gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="tripPositive", marker=baseMarker))

# lag3Tim3Pos
xSub, ySub, zSub = x[x < gate['x']], y[y >= gate['y']], z[z >= gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="lag3Tim3Pos", marker=baseMarker))

# lag3Pd1Pos
xSub, ySub, zSub = x[x >= gate['x']], y[y < gate['y']], z[z >= gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="lag3Pd1Pos", marker=baseMarker))

# pd1Tim3Pos
xSub, ySub, zSub = x[x >= gate['x']], y[y >= gate['y']], z[z < gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="pd1Tim3Pos", marker=baseMarker))

# pd1Pos
xSub, ySub, zSub = x[x >= gate['x']], y[y < gate['y']], z[z < gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="pd1Pos", marker=baseMarker))

# tim3Pos
xSub, ySub, zSub = x[x < gate['x']], y[y >= gate['y']], z[z < gate['z']] 
figData.append(xyzPlot(xSub, ySub, zSub, name="tim3Pos", marker=baseMarker))

figTitle = Path(fcsBaseFn).stem

figLayout = go.Layout(
    title=figTitle,
    width=1024,
    height=1024,
    font=dict(family='".SFNSDisplay-Regular", -apple-system, BlinkMacSystemFont, "Helvetica Neue", "Helvetica", "Calibri", Arial, sans-serif', size=16, color='#000'),
    scene = dict(
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=-1.25, y=-1.15, z=1.8)
        ),
        xaxis = dict(title=xCol.name, range = xRange),
        yaxis = dict(title=yCol.name, range = yRange),
        zaxis = dict(title=zCol.name, range = zRange),
        aspectmode = 'cube')
)

fig = go.FigureWidget(data=figData, layout=figLayout)
fig

FigureWidget({
    'data': [{'marker': {'color': 'rgb(229, 43, 80)',
                         'line': {'color'…

In [5]:
# get reference to camera eye object
eye = fig.layout.scene.camera.eye

In [6]:
# print initial camera eye position
print(eye.x, eye.y, eye.z)

-1.25 -1.15 1.8


In [7]:
# print initial camera eye position
print(eye.x, eye.y, eye.z)

-1.25 -1.15 1.8


In [8]:
# Commit changes back to object
eye.x = eye.x
eye.y = eye.y
eye.z = eye.z

### HTML output looks fine

In [9]:
# Save figure to html file with custom view
plot(fig, filename='/tmp/3d_rotated.html', auto_open=False)

'file:///tmp/3d_rotated.html'

### SVG output appears corrupted to Adobe Illustrator 22.1, but can be opened in Chrome v70.0.3538.77 (albeit with the specified rendering error)

In [10]:
pio.write_image(fig, '/tmp/3d_rotated.svg')

### Other format exports raise errors

In [11]:
pio.write_image(fig, '/tmp/3d_rotated.pdf', validate=False)

ValueError: 
The image request was rejected by the orca conversion utility
with the following error:
   525: plotly.js error
